# Test with Anna KaRNNa to run a sequence from 1 a 12




In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

set the list

In [2]:
list_numbers = np.array([1,2,3,4,5,6,7,8,9,10,11,12])

In [3]:
len(list_numbers)

12

## Making training mini-batches

Return training batchs as a window enter [[1 2 3],[7,8,9]] return [[2,3,4],[8,9,10]] 

In [4]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr_new= arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr_new = arr_new.reshape((n_seqs, -1))
    
    for n in range(0, arr_new.shape[1], n_steps):
        # The features
        x = arr_new[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        #y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        # faz todos menos o ultimo elemento
        y[:, :-1] = x[:, 1:]
        

        # go on the original array 
        y_temp = np.zeros([n_seqs, ], dtype=np.int32)
        for each_seq in range(n_seqs): 
            #idx = n+(n_steps * ((each_seq *  n_seqs )+1))
            idx = n + n_steps + (each_seq * n_batches * n_steps  )
            
            if idx < len( arr ):
                y_temp[each_seq,]= arr[ idx ]
            else:
                y_temp[each_seq,]= arr[ 0 ]
        
        y[:, -1] = y_temp
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [5]:
batches = get_batches(list_numbers, 2, 2)
x, y = next(batches)

In [6]:
print('x\n', x)
print('\ny\n', y)

x
 [[1 2]
 [7 8]]

y
 [[2 3]
 [8 9]]


In [7]:
x, y = next(batches)
print('x\n', x)
print('\ny\n', y)

x
 [[ 3  4]
 [ 9 10]]

y
 [[ 4  5]
 [10 11]]


## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`.

In [8]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. For example,

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow will create different weight matrices for all `cell` objects. Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [9]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$.

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

In [10]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

In [11]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [12]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [13]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=2, num_steps=3, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
        


## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [19]:
batch_size = 2       # Sequences per batch
num_steps = 3      # Number of sequence steps per batch
lstm_size = 256      # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.



In [20]:
epochs = 800

model = CharRNN(len(list_numbers), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)


saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(list_numbers, batch_size, num_steps):
            counter += 1
            start = time.time()
  
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)

            end = time.time()





            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
    

    
    saver.save(sess, "checkpoints/resultado.ckpt")




Epoch: 1/800...  Training Step: 1...  Training loss: 2.4900...  0.0376 sec/batch
Epoch: 1/800...  Training Step: 2...  Training loss: 2.0769...  0.0087 sec/batch
Epoch: 2/800...  Training Step: 3...  Training loss: 2.4765...  0.0082 sec/batch
Epoch: 2/800...  Training Step: 4...  Training loss: 2.0619...  0.0084 sec/batch
Epoch: 3/800...  Training Step: 5...  Training loss: 2.4649...  0.0086 sec/batch
Epoch: 3/800...  Training Step: 6...  Training loss: 2.0610...  0.0085 sec/batch
Epoch: 4/800...  Training Step: 7...  Training loss: 2.4642...  0.0091 sec/batch
Epoch: 4/800...  Training Step: 8...  Training loss: 2.0468...  0.0088 sec/batch
Epoch: 5/800...  Training Step: 9...  Training loss: 2.4575...  0.0087 sec/batch
Epoch: 5/800...  Training Step: 10...  Training loss: 2.0195...  0.0085 sec/batch
Epoch: 6/800...  Training Step: 11...  Training loss: 2.4473...  0.0084 sec/batch
Epoch: 6/800...  Training Step: 12...  Training loss: 2.0184...  0.0087 sec/batch
Epoch: 7/800...  Training

Epoch: 55/800...  Training Step: 109...  Training loss: 1.0543...  0.0095 sec/batch
Epoch: 55/800...  Training Step: 110...  Training loss: 0.2694...  0.0093 sec/batch
Epoch: 56/800...  Training Step: 111...  Training loss: 1.1177...  0.0091 sec/batch
Epoch: 56/800...  Training Step: 112...  Training loss: 0.0990...  0.0089 sec/batch
Epoch: 57/800...  Training Step: 113...  Training loss: 1.0723...  0.0086 sec/batch
Epoch: 57/800...  Training Step: 114...  Training loss: 0.1672...  0.0087 sec/batch
Epoch: 58/800...  Training Step: 115...  Training loss: 1.0837...  0.0089 sec/batch
Epoch: 58/800...  Training Step: 116...  Training loss: 0.2581...  0.0083 sec/batch
Epoch: 59/800...  Training Step: 117...  Training loss: 0.9446...  0.0087 sec/batch
Epoch: 59/800...  Training Step: 118...  Training loss: 0.3027...  0.0084 sec/batch
Epoch: 60/800...  Training Step: 119...  Training loss: 0.9872...  0.0087 sec/batch
Epoch: 60/800...  Training Step: 120...  Training loss: 0.1376...  0.0085 se

Epoch: 110/800...  Training Step: 219...  Training loss: 0.3531...  0.0090 sec/batch
Epoch: 110/800...  Training Step: 220...  Training loss: 0.0208...  0.0098 sec/batch
Epoch: 111/800...  Training Step: 221...  Training loss: 0.2800...  0.0097 sec/batch
Epoch: 111/800...  Training Step: 222...  Training loss: 0.0150...  0.0087 sec/batch
Epoch: 112/800...  Training Step: 223...  Training loss: 0.2637...  0.0089 sec/batch
Epoch: 112/800...  Training Step: 224...  Training loss: 0.0915...  0.0089 sec/batch
Epoch: 113/800...  Training Step: 225...  Training loss: 0.2644...  0.0087 sec/batch
Epoch: 113/800...  Training Step: 226...  Training loss: 0.1097...  0.0088 sec/batch
Epoch: 114/800...  Training Step: 227...  Training loss: 0.2276...  0.0085 sec/batch
Epoch: 114/800...  Training Step: 228...  Training loss: 0.0212...  0.0086 sec/batch
Epoch: 115/800...  Training Step: 229...  Training loss: 0.2173...  0.0085 sec/batch
Epoch: 115/800...  Training Step: 230...  Training loss: 0.0109..

Epoch: 166/800...  Training Step: 331...  Training loss: 0.0334...  0.0096 sec/batch
Epoch: 166/800...  Training Step: 332...  Training loss: 0.0198...  0.0099 sec/batch
Epoch: 167/800...  Training Step: 333...  Training loss: 0.0643...  0.0092 sec/batch
Epoch: 167/800...  Training Step: 334...  Training loss: 0.0143...  0.0089 sec/batch
Epoch: 168/800...  Training Step: 335...  Training loss: 0.0675...  0.0083 sec/batch
Epoch: 168/800...  Training Step: 336...  Training loss: 0.0546...  0.0087 sec/batch
Epoch: 169/800...  Training Step: 337...  Training loss: 0.0309...  0.0087 sec/batch
Epoch: 169/800...  Training Step: 338...  Training loss: 0.0048...  0.0086 sec/batch
Epoch: 170/800...  Training Step: 339...  Training loss: 0.0441...  0.0087 sec/batch
Epoch: 170/800...  Training Step: 340...  Training loss: 0.0178...  0.0088 sec/batch
Epoch: 171/800...  Training Step: 341...  Training loss: 0.0424...  0.0084 sec/batch
Epoch: 171/800...  Training Step: 342...  Training loss: 0.0040..

Epoch: 221/800...  Training Step: 442...  Training loss: 0.0041...  0.0093 sec/batch
Epoch: 222/800...  Training Step: 443...  Training loss: 0.0223...  0.0099 sec/batch
Epoch: 222/800...  Training Step: 444...  Training loss: 0.0104...  0.0088 sec/batch
Epoch: 223/800...  Training Step: 445...  Training loss: 0.0140...  0.0088 sec/batch
Epoch: 223/800...  Training Step: 446...  Training loss: 0.0020...  0.0088 sec/batch
Epoch: 224/800...  Training Step: 447...  Training loss: 0.0646...  0.0084 sec/batch
Epoch: 224/800...  Training Step: 448...  Training loss: 0.0070...  0.0088 sec/batch
Epoch: 225/800...  Training Step: 449...  Training loss: 0.0159...  0.0088 sec/batch
Epoch: 225/800...  Training Step: 450...  Training loss: 0.0090...  0.0088 sec/batch
Epoch: 226/800...  Training Step: 451...  Training loss: 0.0195...  0.0087 sec/batch
Epoch: 226/800...  Training Step: 452...  Training loss: 0.0265...  0.0084 sec/batch
Epoch: 227/800...  Training Step: 453...  Training loss: 0.0194..

...

Epoch: 554/800...  Training Step: 1108...  Training loss: 0.0009...  0.0101 sec/batch
Epoch: 555/800...  Training Step: 1109...  Training loss: 0.0028...  0.0105 sec/batch
Epoch: 555/800...  Training Step: 1110...  Training loss: 0.0017...  0.0090 sec/batch
Epoch: 556/800...  Training Step: 1111...  Training loss: 0.0018...  0.0087 sec/batch
Epoch: 556/800...  Training Step: 1112...  Training loss: 0.0017...  0.0086 sec/batch
Epoch: 557/800...  Training Step: 1113...  Training loss: 0.0018...  0.0088 sec/batch
Epoch: 557/800...  Training Step: 1114...  Training loss: 0.0010...  0.0083 sec/batch
Epoch: 558/800...  Training Step: 1115...  Training loss: 0.0016...  0.0087 sec/batch
Epoch: 558/800...  Training Step: 1116...  Training loss: 0.0012...  0.0087 sec/batch
Epoch: 559/800...  Training Step: 1117...  Training loss: 0.0036...  0.0084 sec/batch
Epoch: 559/800...  Training Step: 1118...  Training loss: 0.0006...  0.0087 sec/batch
Epoch: 560/800...  Training Step: 1119...  Training lo

Epoch: 609/800...  Training Step: 1218...  Training loss: 0.0014...  0.0092 sec/batch
Epoch: 610/800...  Training Step: 1219...  Training loss: 0.0020...  0.0094 sec/batch
Epoch: 610/800...  Training Step: 1220...  Training loss: 0.0030...  0.0088 sec/batch
Epoch: 611/800...  Training Step: 1221...  Training loss: 0.0007...  0.0087 sec/batch
Epoch: 611/800...  Training Step: 1222...  Training loss: 0.0014...  0.0088 sec/batch
Epoch: 612/800...  Training Step: 1223...  Training loss: 0.0009...  0.0084 sec/batch
Epoch: 612/800...  Training Step: 1224...  Training loss: 0.0030...  0.0080 sec/batch
Epoch: 613/800...  Training Step: 1225...  Training loss: 0.0022...  0.0081 sec/batch
Epoch: 613/800...  Training Step: 1226...  Training loss: 0.0044...  0.0084 sec/batch
Epoch: 614/800...  Training Step: 1227...  Training loss: 0.0013...  0.0085 sec/batch
Epoch: 614/800...  Training Step: 1228...  Training loss: 0.0017...  0.0085 sec/batch
Epoch: 615/800...  Training Step: 1229...  Training lo

Epoch: 665/800...  Training Step: 1329...  Training loss: 0.0019...  0.0093 sec/batch
Epoch: 665/800...  Training Step: 1330...  Training loss: 0.0010...  0.0103 sec/batch
Epoch: 666/800...  Training Step: 1331...  Training loss: 0.0006...  0.0090 sec/batch
Epoch: 666/800...  Training Step: 1332...  Training loss: 0.0025...  0.0083 sec/batch
Epoch: 667/800...  Training Step: 1333...  Training loss: 0.0018...  0.0084 sec/batch
Epoch: 667/800...  Training Step: 1334...  Training loss: 0.0033...  0.0087 sec/batch
Epoch: 668/800...  Training Step: 1335...  Training loss: 0.0046...  0.0086 sec/batch
Epoch: 668/800...  Training Step: 1336...  Training loss: 0.0017...  0.0088 sec/batch
Epoch: 669/800...  Training Step: 1337...  Training loss: 0.0010...  0.0087 sec/batch
Epoch: 669/800...  Training Step: 1338...  Training loss: 0.0017...  0.0088 sec/batch
Epoch: 670/800...  Training Step: 1339...  Training loss: 0.0020...  0.0086 sec/batch
Epoch: 670/800...  Training Step: 1340...  Training lo

Epoch: 721/800...  Training Step: 1441...  Training loss: 0.0027...  0.0085 sec/batch
Epoch: 721/800...  Training Step: 1442...  Training loss: 0.0030...  0.0102 sec/batch
Epoch: 722/800...  Training Step: 1443...  Training loss: 0.0011...  0.0086 sec/batch
Epoch: 722/800...  Training Step: 1444...  Training loss: 0.0028...  0.0087 sec/batch
Epoch: 723/800...  Training Step: 1445...  Training loss: 0.0019...  0.0087 sec/batch
Epoch: 723/800...  Training Step: 1446...  Training loss: 0.0008...  0.0084 sec/batch
Epoch: 724/800...  Training Step: 1447...  Training loss: 0.0043...  0.0088 sec/batch
Epoch: 724/800...  Training Step: 1448...  Training loss: 0.0013...  0.0086 sec/batch
Epoch: 725/800...  Training Step: 1449...  Training loss: 0.0007...  0.0086 sec/batch
Epoch: 725/800...  Training Step: 1450...  Training loss: 0.0005...  0.0084 sec/batch
Epoch: 726/800...  Training Step: 1451...  Training loss: 0.0023...  0.0085 sec/batch
Epoch: 726/800...  Training Step: 1452...  Training lo

Epoch: 776/800...  Training Step: 1552...  Training loss: 0.0022...  0.0083 sec/batch
Epoch: 777/800...  Training Step: 1553...  Training loss: 0.0019...  0.0091 sec/batch
Epoch: 777/800...  Training Step: 1554...  Training loss: 0.0010...  0.0095 sec/batch
Epoch: 778/800...  Training Step: 1555...  Training loss: 0.0015...  0.0082 sec/batch
Epoch: 778/800...  Training Step: 1556...  Training loss: 0.0019...  0.0083 sec/batch
Epoch: 779/800...  Training Step: 1557...  Training loss: 0.0024...  0.0080 sec/batch
Epoch: 779/800...  Training Step: 1558...  Training loss: 0.0006...  0.0078 sec/batch
Epoch: 780/800...  Training Step: 1559...  Training loss: 0.0006...  0.0081 sec/batch
Epoch: 780/800...  Training Step: 1560...  Training loss: 0.0008...  0.0083 sec/batch
Epoch: 781/800...  Training Step: 1561...  Training loss: 0.0008...  0.0084 sec/batch
Epoch: 781/800...  Training Step: 1562...  Training loss: 0.0011...  0.0082 sec/batch
Epoch: 782/800...  Training Step: 1563...  Training lo

In [25]:
   
def sample(checkpoint,lstm_size, vocab_size,  batch_size=1, num_steps=1, inputs=None, init_state=None):
    model = CharRNN(len(list_numbers), lstm_size=lstm_size, sampling=False, batch_size=batch_size, num_steps=num_steps)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        if init_state == None:
            new_state = sess.run(model.initial_state)
        else: 
            new_state = init_state
            
        feed = {model.inputs: inputs,
                model.keep_prob: 1.,
                model.initial_state: new_state}
        preds, new_state = sess.run([model.prediction, model.final_state], 
                                     feed_dict=feed)
        print("Input:\n",inputs)
        print('Result\n',np.argmax(preds, axis=1)) 
        print("\n")
        return new_state

In [21]:
checkpoint = tf.train.latest_checkpoint('checkpoints')


In [35]:
# predict next number
batchSize = 2
numSteps = 3
x = np.zeros([batchSize,numSteps])

# send 1st training batch
x = np.array([[1,2,3],[7,8,9]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

# send 2nd training batch
x = np.array([[4,5,6],[10,11,12]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

# send mixed sequ
x = np.array([[3,4,5],[6,7,8]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)


# different small size
batchSize = 2
numSteps = 2
x = np.zeros([batchSize,numSteps])

# send 1st training batch
x = np.array([[1,2],[7,8]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

# send 2nd training batch
x = np.array([[4,5],[10,11]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

# send mixed sequ
x = np.array([[3,4],[9,10]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)


# different bigger size
batchSize = 2
numSteps = 6
x = np.zeros([batchSize,numSteps])

# send 1st training batch
x = np.array([[1,2,3,4,5,6],[7,8,9,10,11,12]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

# different bigger size
batchSize = 1
numSteps = 6
x = np.zeros([batchSize,numSteps])

# send 1st training batch
x = np.array([[1,2,3,4,5,6]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

x = np.array([[7,8,9,10,11,12]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

x = np.array([[3,4,5,6,7,8]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

# different bigger size
batchSize = 1
numSteps = 2
x = np.zeros([batchSize,numSteps])

# send 1st training batch
x = np.array([[1,2]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

x = np.array([[7,8]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

x = np.array([[3,4]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

x = np.array([[2,3]])
sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)


# different bigger size
batchSize = 1
numSteps = 1
x = np.zeros([batchSize,numSteps])

# without the init state
print("Test without passing the init state")
for test_number in range(1,13):
    x = np.array([[test_number]])
    sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x)

print("Test passing the init state")
for test_number in range(1,13):
    x = np.array([[test_number]])
    if test_number == 1:
        ini_state = None
    ini_state = sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x, init_state=ini_state )

# the results above shows that the initstate works for each batch row if we pass the initstate from the first batch row 
# to the second batch row it gets confuse in our teste 2x3 first row is [1 2 3 4 5 6] and second is [7 8 9 10 11 12]
# 

# here I teste with passing the init state for each row
print("Test passing the init state only for each row")
for test_number in range(1,13):
    x = np.array([[test_number]])
    if test_number == 1 or test_number == 7:
        ini_state = None
    ini_state = sample(checkpoint, lstm_size, len(list_numbers), batch_size=batchSize, num_steps=numSteps, inputs=x, init_state=ini_state )


Input:
 [[1 2 3]
 [7 8 9]]
Result
 [ 2  3  4  8  9 10]


Input:
 [[ 4  5  6]
 [10 11 12]]
Result
 [8 8 9 2 3 4]


Input:
 [[3 4 5]
 [6 7 8]]
Result
 [8 9 5 8 8 9]


Input:
 [[1 2]
 [7 8]]
Result
 [2 3 8 9]


Input:
 [[ 4  5]
 [10 11]]
Result
 [8 8 2 3]


Input:
 [[ 3  4]
 [ 9 10]]
Result
 [8 9 8 3]


Input:
 [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]]
Result
 [ 2  3  4  5  6  7  8  9 10 11  1  1]


Input:
 [[1 2 3 4 5 6]]
Result
 [2 3 4 5 6 7]


Input:
 [[ 7  8  9 10 11 12]]
Result
 [ 8  9 10 11  1  1]


Input:
 [[3 4 5 6 7 8]]
Result
 [8 9 5 6 6 7]


Input:
 [[1 2]]
Result
 [2 3]


Input:
 [[7 8]]
Result
 [8 9]


Input:
 [[3 4]]
Result
 [8 9]


Input:
 [[2 3]]
Result
 [2 3]


Test without passing the init state
Input:
 [[1]]
Result
 [2]


Input:
 [[2]]
Result
 [2]


Input:
 [[3]]
Result
 [8]


Input:
 [[4]]
Result
 [8]


Input:
 [[5]]
Result
 [8]


Input:
 [[6]]
Result
 [8]


Input:
 [[7]]
Result
 [8]


Input:
 [[8]]
Result
 [8]


Input:
 [[9]]
Result
 [8]


Input:
 [[10]]
Result
 [2]
